In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score, make_scorer

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix, log_loss
import scipy.stats as st

np.random.seed(42)

In [2]:
def get_cm(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    cm = cm / cm.astype(float).sum(axis=1)
    np.round_(cm, decimals=4, out=cm)
    
    return cm

In [3]:
def deskew_df(df):
    box_cox_df = pd.DataFrame()
    for col in df.columns:
        box_cox_col = st.boxcox(df[col])[0]
        box_cox_df[col] = pd.Series(box_cox_col)
        
    return box_cox_df

In [4]:
def test_model(my_model, X_train, y_train, X_test, y_test, my_rand_state=42, pca_flag=False, gs_params=None):
    if pca_flag:
        pipe = Pipeline([('scaler', StandardScaler()),
                         ('pca', PCA(n_components=5, random_state=my_rand_state)),
                         ('clf', my_model)
                        ])
    else:
        pipe = Pipeline([('scaler', StandardScaler()),
                         ('clf', my_model)
                        ])
    
    if gs_params != None:
        roc_auc_scorer = make_scorer(roc_auc_score, greater_is_better=True)
        pipe_gs = GridSearchCV(pipe, param_grid=gs_params, cv=10, n_jobs=-1, verbose=1, scoring=roc_auc_scorer)
        pipe = pipe_gs
    
    pipe.fit(X_train, y_train)    
    train_score = pipe.score(X_train, y_train)
    test_score = pipe.score(X_test, y_test)
    
    y_pred = pipe.predict(X_test)
    cm = get_cm(y_test, y_pred)
    ll = log_loss(y_test, y_pred)
    
    print(my_model)
    print('train score:', train_score)
    print('test score:', test_score)
    print('log loss:', ll)
    print('confusion matrix:\n', cm)
    print()
    
    if gs_params != None:
        return pipe.best_params_

In [5]:
m_features = pd.read_pickle('../pickled_data-UCI/madelon_important_features.p')
imp_feats = list(m_features['feature'].head(20))

In [6]:
X_train = pd.read_pickle('../pickled_data-UCI/sample_1/df_sample_data.p')
X_train = X_train[imp_feats]

y_train = pd.read_pickle('../pickled_data-UCI/sample_1/df_sample_labels.p')
y_train = y_train.values.ravel()

X_test = pd.read_pickle('../pickled_data-UCI/madelon_valid_data.p')
X_test = X_test[imp_feats]

y_test = pd.read_pickle('../pickled_data-UCI/madelon_valid_labels.p')
y_test = y_test.values.ravel()

In [7]:
mm_scaler = MinMaxScaler(feature_range=(0.00001, 1))

X_train_mmsc = mm_scaler.fit_transform(X_train)
X_test_mmsc = mm_scaler.fit_transform(X_test)

In [8]:
X_train_ds = deskew_df(pd.DataFrame(X_train_mmsc))
X_test_ds = deskew_df(pd.DataFrame(X_test_mmsc))

## Use GridSearchCV with Feature Selection

In [9]:
lr = LogisticRegression(random_state=42, n_jobs=-1)
lr_params = {
    'clf__penalty' : ['l1','l2'],
    'clf__C' : np.logspace(-10,-1,10)
}

test_model(lr, X_train_ds, y_train, X_test_ds, y_test, gs_params=lr_params)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
train score: 0.625016500594
test score: 0.565
log loss: 15.0244423609
confusion matrix:
 [[ 0.8133  0.1867]
 [ 0.6833  0.3167]]



[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    1.2s finished


{'clf__C': 0.10000000000000001, 'clf__penalty': 'l1'}

In [10]:
dtc = DecisionTreeClassifier(random_state=42)
dtc_params = {
    'clf__criterion' : ['gini','entropy'],
    'clf__class_weight' : ['balanced', None]
}

test_model(dtc, X_train_ds, y_train, X_test_ds, y_test, gs_params=dtc_params)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best')
train score: 1.0
test score: 0.696666666667
log loss: 10.476904768
confusion matrix:
 [[ 0.6433  0.3567]
 [ 0.25    0.75  ]]



[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.4s finished


{'clf__class_weight': None, 'clf__criterion': 'gini'}

In [11]:
knc = KNeighborsClassifier(n_jobs=-1)
knc_params = {
    'clf__n_neighbors' : [x * 10 for x in range(1,9)],
    'clf__weights' : ['uniform', 'distance'],
    'clf__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'clf__p': [1, 2, 3]
}

test_model(knc, X_train_ds, y_train, X_test_ds, y_test, gs_params=knc_params)

Fitting 10 folds for each of 192 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   58.2s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  6.5min
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklear

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklear

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklear

...

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklear

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklear

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklear

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')
train score: 1.0
test score: 0.74
log loss: 8.98017248372
confusion matrix:
 [[ 0.7733  0.2267]
 [ 0.2933  0.7067]]



{'clf__algorithm': 'auto',
 'clf__n_neighbors': 10,
 'clf__p': 1,
 'clf__weights': 'distance'}

In [12]:
my_svc = SVC(random_state=42)
svc_params = {
    'clf__C' : np.logspace(-10,-1,10),
    'clf__kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
    'clf__degree' : [2, 3]
}

test_model(my_svc, X_train_ds, y_train, X_test_ds, y_test, gs_params=svc_params)

Fitting 10 folds for each of 80 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:   32.1s finished


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)
train score: 0.6943249957
test score: 0.656666666667
log loss: 11.8583758641
confusion matrix:
 [[ 0.8433  0.1567]
 [ 0.53    0.47  ]]



{'clf__C': 0.10000000000000001, 'clf__degree': 2, 'clf__kernel': 'rbf'}

## Use GridSearchCV with Feature Selection and Dimensionality Reduction

In [13]:
test_model(lr, X_train_ds, y_train, X_test_ds, y_test, pca_flag=True, gs_params=lr_params)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
train score: 0.58295498638
test score: 0.575
log loss: 14.6791825325
confusion matrix:
 [[ 0.4933  0.5067]
 [ 0.3433  0.6567]]



[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    1.9s finished


{'clf__C': 0.01, 'clf__penalty': 'l1'}

In [14]:
test_model(dtc, X_train_ds, y_train, X_test_ds, y_test, pca_flag=True, gs_params=dtc_params)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best')
train score: 1.0
test score: 0.663333333333
log loss: 11.6281826552
confusion matrix:
 [[ 0.68    0.32  ]
 [ 0.3533  0.6467]]



[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.7s finished


{'clf__class_weight': 'balanced', 'clf__criterion': 'entropy'}

In [15]:
test_model(knc, X_train_ds, y_train, X_test_ds, y_test, pca_flag=True, gs_params=knc_params)

Fitting 10 folds for each of 192 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   56.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  2.8min
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklear

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklear

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklear

...

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklear

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklear

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/opt/conda/lib/python3.6/site-packages/sklear

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')
train score: 1.0
test score: 0.721666666667
log loss: 9.61341670085
confusion matrix:
 [[ 0.69    0.31  ]
 [ 0.2467  0.7533]]



{'clf__algorithm': 'auto',
 'clf__n_neighbors': 10,
 'clf__p': 1,
 'clf__weights': 'distance'}

In [16]:
test_model(my_svc, X_train_ds, y_train, X_test_ds, y_test, pca_flag=True, gs_params=svc_params)

Fitting 10 folds for each of 80 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:   13.8s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)
train score: 0.74225472117
test score: 0.72
log loss: 9.67096400357
confusion matrix:
 [[ 0.7333  0.2667]
 [ 0.2933  0.7067]]



[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:   42.9s finished


{'clf__C': 0.10000000000000001, 'clf__degree': 2, 'clf__kernel': 'rbf'}